In [60]:
from datasets import load_from_disk

SEED = 123
FOLDER = "../data/raw/legal-qa-v1"

In [61]:
dataset = load_from_disk(FOLDER)

In [62]:
dataset["answer"]

Column(["A:In Kentucky, your situation raises questions about patient rights and medical records access. If you were discharged from a pain management office and subsequently lost access to your patient portal, it's important to understand your rights regarding medical records. Under the Health Insurance Portability and Accountability Act (HIPAA), you have the right to access your own medical records. A healthcare provider cannot deny you access to your records, even if you were discharged from their practice. The issue with the patient portal could be a technical glitch or a deliberate action, but either way, it needs to be addressed. You should contact the pain management office directly to inquire about the issue with the patient portal. They are obligated to provide you with a way to access your medical records, which may include providing paper copies or access through a different electronic means. If the office is unresponsive or refuses to provide access to your records, conside

In [63]:
def transform(example):
    # ejemplo: poner preguntas en mayúsculas y respuestas en minúsculas
    new_question = example["question"].replace("Q:", "").strip()
    new_answer = example["answer"].replace("A:", "").strip()

    return {
        "question": new_question,
        "answer": new_answer
    }

In [64]:
transformed_dataset = dataset.map(transform)

In [65]:
print(transformed_dataset["question"][0])
print(transformed_dataset["answer"][0])

I was wondering if a pain management office is acting illegally/did an illegal action.. I was discharged as a patient from a pain management office after them telling me that a previous pain management specialist I saw administered a steroid shot wrong and I told them in the portal that I spoke to lawyers for advice but no lawsuit/case was created. It was maybe 1-2 months after I was discharged that I no longer have access to my patient portal with them. Every time I try to login I enter my credentials, wait a few seconds, and then I get re-directed back to the original screen where I have various options to login. I know I can speak to the office directly and ask them about what specifically is going on, talk to other lawyers if this is a violation of my rights, etc. but I was just wondering if anyone on this site would know if this action is in fact illegal.
In Kentucky, your situation raises questions about patient rights and medical records access. If you were discharged from a pai

In [66]:
# split the dataset into train and validation
train_test = transformed_dataset.train_test_split(test_size=0.2, seed=SEED)
print(train_test)
train_val = train_test["test"].train_test_split(test_size=0.5, seed=SEED)
transformed_dataset = {
    "train": train_test["train"],
    "val": train_val["test"],
    "test": train_val["train"]
}

DatasetDict({
    train: Dataset({
        features: ['question', 'answer'],
        num_rows: 2993
    })
    test: Dataset({
        features: ['question', 'answer'],
        num_rows: 749
    })
})


In [67]:
transformed_dataset

{'train': Dataset({
     features: ['question', 'answer'],
     num_rows: 2993
 }),
 'val': Dataset({
     features: ['question', 'answer'],
     num_rows: 375
 }),
 'test': Dataset({
     features: ['question', 'answer'],
     num_rows: 374
 })}